
<div align="center">
  <h1></h1>
  <h1>Multi-Agent Meal Planning</h1>
  <h4 align="center">Assignmnet III</h4>
  <h4 align="center"></h4>
</div>

Welcome to Assignment III!

In this notebook, you will design and implement a Multi-Agent Meal Planning System that generates meal recommendations while adhering to budgetary and nutritional constraints. Using a team of specialized agents, you'll create a dynamic and interactive system capable of tailoring meal plans to diverse user requirements.

**By the end of this assignment, you'll be able to:**
1. Build a Multi-Agent System to collaboratively solve a complex task, such as
meal planning.
2. Design agents with specialized roles, such as recommending meals or validating budgets.
3. Implement agent coordination mechanisms using RoundRobinGroupChat for group chatting.
4. Integrate external tools (e.g., a Budget Checker Tool) to ensure the system adheres to financial constraints.
5. Use termination methods effectively to manage agent interactions and prevent resource overuse.

## Group Members


1. First memebr:
  * Name: Aman Sudhakar Bhanse
  * Matrikel-Nr.: 430874
2. Second memebr:
  * Name: Sourav Chitnis
  * Matrikel-Nr.: 430876
2. Third memebr:
  * Name:
  * Matrikel-Nr.:

# 1. Access to GROQ.
Execute the following cell to connect to your Groq account.

In [26]:
import os
from getpass import getpass

tokenGROQ = getpass('Enter GROQ_API_KEY here: ')
os.environ["GROQ_API_KEY"] = tokenGROQ

# 2. Packages
Execute the following code cells for installing the packages needed.

In [27]:
!pip install autogen-agentchat autogen-ext[openai] matplotlib yfinance -q
!pip install groq -q


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# 3. Problem Statement: Multi-Agent Meal Planning System with Budget Management


In this notebook we will design and implement a multi-agent system to assist users in meal planning while addressing the following:

* Dietary preferences: Personalization based on restrictions and user preferences.
* Nutritional goals: Balanced and healthy meal recommendations.
* Financial constraints: Ensuring affordability through budget validation.

This system will include a Budget Checker Tool to guarantee all recommendations fit within the user's financial limits.


**System**

1. Personalized Meal Recommendations:

* Provides tailored meal suggestions for breakfast, lunch, dinner, and snacks.
* Considers dietary restrictions, time availability, and individual preferences.
2. Budget Validation:

* Uses the Budget Checker Tool to confirm that meal costs fit within the user’s budget.
3. User-Friendly Assistance:

* Delivers clear, concise, and actionable recommendations for each meal type.

**Agent Roles**

1. Breakfast Agent:

Role: Recommends breakfast options that align with dietary preferences, preparation time, and nutritional goals.

Specialization: Provides a single breakfast suggestion and validates its affordability using the Budget Checker Tool.
2. Lunch Agent:

Role: Suggests balanced and nutritious lunch options tailored to dietary needs.

Specialization: Offers one lunch suggestion and ensures it fits within the budget.

3. Dinner Agent:

Role: Recommends practical dinner options based on dietary requirements, budget, and preparation constraints.

Specialization: Validates that dinner suggestions are both feasible and affordable.

4. Snack Agent:

Role: Proposes healthy and satisfying snack options to complement the day’s meals.

Specialization: Verifies that snack recommendations are affordable and nutritionally balanced.

5. Budget Agent:

Role: Ensures all meal costs remain within the user's remaining budget by using the Budget Checker Tool.

Specialization: Provides real-time budget feedback to guide meal selections and prevent overspending.



**Custom Tool: Budget Checker Tool**
1. Purpose:

* Validates if a proposed meal cost fits within the user’s current budget.

2. Features:

* Budget Validation: Compares the cost of a meal against the remaining budget.
* Feedback: Provides clear, actionable messages indicating:

  Whether the cost is affordable.

  How much budget remains or is exceeded.

* Integration: Used by all meal-specific agents to ensure recommendations are financially feasible.

# 4. Configure the Model Client:

Use the OpenAIChatCompletionClient class to create an instance of the model client.

Configure it with the following:

* Model name: "llama3-70b-8192".

* API base URL: "https://api.groq.com/openai/v1".

* API key: Use an environment variable named "GROQ_API_KEY" for secure authentication.

* Model settings: Include attributes like whether vision is enabled, function-calling support, and JSON output format.

In [28]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Step 1: Create the model client
# Complete the following implementation
model_client = OpenAIChatCompletionClient(
    model="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "llama3",
    },
)

# Step 2: Verify the client is set up correctly
print(model_client)

# 5. Budget Tracker Tool

The provided code implements a Budget Checker Tool to validate whether a given cost is within the current budget.

In [29]:
from autogen_core.tools import FunctionTool

def budget_checker_tool(cost: float, current_budget: float) -> dict:
    """
    Check if a given cost fits within the current budget.

    :param cost: float, the cost to check.
    :param current_budget: float, the current remaining budget.
    :return: dict, containing whether the cost fits the budget and a message.
    """
    if cost > current_budget:
        return {
            "fits_budget": False,
            "message": f"This exceeds your budget by ${cost - current_budget:.2f}."
        }
    return {
        "fits_budget": True,
        "message": f"The cost of ${cost:.2f} fits within your budget."
    }

# Example usage with the FunctionTool
budget_checker_tool = FunctionTool(
    budget_checker_tool,
    description="Check if a given cost fits within the current budget."
)

#6. Agent Design

In this task, you will implement a set of specialized Assistant Agents to collaboratively plan meals while addressing dietary, nutritional, and budgetary constraints. You will design Meal Planner Agents for breakfast, lunch, dinner, and snacks, each with clear roles and behavioral instructions.

In [30]:
# Import the required class
from autogen_agentchat.agents import AssistantAgent

# Step 1: Define the Budget Agent
# Complete the following implementation
budget_agent = AssistantAgent(
    name="BudgetAgent",
    tools=[budget_checker_tool],  # Use the budget checker tool
    model_client=model_client,
    description="An agent that checks meal costs are within the allocated budget.",
    system_message="You are an helpful AI assistant that ensures all the meal recommendations are within the budget provided."
)

# Step 2: Define the Breakfast Planner Agent
# Complete the following implementation
breakfast_agent = AssistantAgent(
    name="BreakfastAgent",
    model_client=model_client,
    description="An agent that recommends breakfast options that align with dietary preferences, preparation time, and nutritional goals.",
    system_message="You are an helpful AI assistant that provides a single breakfast suggestion that is quick, healthy and budget friendly."
)

# Step 3: Define the Lunch Planner Agent
# Complete the following implementation
lunch_agent = AssistantAgent(
    name="LunchAgent",
    model_client=model_client,
    description="An agent that suggests balanced and nutritious lunch options tailored to dietary needs and user.",
    system_message="You are an helpful AI assistant that offers one lunch suggestion and ensures it fits within the budget and is healthy."
)

# Step 4: Define the Dinner Planner Agent
# Complete the following implementation
dinner_agent = AssistantAgent(
    name="DinnerAgent",
    model_client=model_client,
    description="An agent that recommends practical dinner options based on dietary requirements, budget, and preparation constraints.",
    system_message="You are an helpful AI assistant that provides one dinner suggestion and validates that it is both feasible and affordable and meets dietary requirements."
)

# Step 5: Define the Snack Planner Agent
# Complete the following implementation
snack_agent = AssistantAgent(
    name="SnackAgent",
    model_client=model_client,
    description="An agent that proposes healthy and satisfying snack options to complement daily meals.",
    system_message="You are an helpful AI assistant that focus on healthy and affordable snack recommendations that fit within the user’s remaining budget."
)

# Step 6: Test the agents
print("Agents initialized successfully!")


Agents initialized successfully!



- Question: which termination methods are available (introduced in the exercise) and what are they best used for and what are the potential issues?

Answer: In this exercise we have used MaxMessageTermination. There are other approaches as well. We have explained all the methods.
1. MaxMessageTermination (we have used in this exercise): 
Use to limit the conversation to a limited message. 
Conversation may not be completed fully before it gets terminated due to low message limit.
2. TextMentionTermination:
Used to end the conversation with specific keyword.
Agents need to know and use the specific termination phrase. Accidental mentions could end the conversation abruptly.
3. TokenUsageTermination:
Used to stop the conversation after a certain number of tokens have been used.
Agents must report token usage accurately, misreporting can lead to premature or delayed termination.
4. TimeoutTermination:
Used to end the conversation after a specified duration to ensure timely responses.
Conversations might be cut off if the time limit is too short.
5. HandoffTermination:
Pausing the conversation to allow external input, such as from a user or another system, when an agent requests a handoff.
It requires proper handling of handoff messages. If not then conversation may not resume correctly.
6. SourceMatchTermination:
Ending the conversation after a specific agent responds.
Depends on the timely response of the specified agent as delays or absence can affect termination.
7. ExternalTermination:
Allows external systems or user interfaces to programmatically control the termination, such as a Stop button in chat.
It requires reliable external controls.
8. StopMessageTermination:
Ending the conversation when an agent produces a StopMessage, indicating completion.
Agents must be designed to generate StopMessages appropriately if not then it can lead to failure to terminate.

- Question: why is it important to have a termination condition?

Answer: Termination requirements are essential for controlling agent-to-agent communications in order to:
Avoid Infinite Loops: Agents may exchange messages forever without termination conditions, which could exhaust available resources.
Ensure Efficiency: Conversations can cease when goals are reached by clearly identifying endpoints, which saves computing power.
Preserve Control: Termination conditions give programmers the ability to control and anticipate how agent interactions will behave, making sure that they produce the intended results.

- Question: Can the approaches be combined?

Answer: Combining termination criteria adds complexity even as it provides flexibility:
Added Complexity: Handling several termination conditions might add complexity to the system and make debugging more challenging.
Possible Conflicts: Unexpected behavior may result from conflicts between various termination criteria. A TimeoutTermination, for example, may terminate a conversation before a TextMentionTermination condition is satisfied.
Maintenance Overhead: Updating and maintaining the termination logic might become more difficult as the number of coupled circumstances rises.


# 7. Implementing Termination
Implement a MaxMessageTermination with 10 max_messages

In [31]:
from autogen_agentchat.conditions import MaxMessageTermination
# Complete the following implementation
termination = MaxMessageTermination(max_messages=10)

#8. RoundRobinGroupChat

In this part, you will implement a RoundRobinGroupChat team to coordinate multiple agents for a meal-planning task. The system will use a sequential approach, ensuring that each agent contributes to the task in a pre-defined order.

please think about the sequence of the agents that you defined above. Which sequence might be best?

In [32]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
# Complete the following implementation
team = RoundRobinGroupChat([breakfast_agent,lunch_agent,snack_agent,dinner_agent,budget_agent],termination_condition=termination)

In [35]:
# Complete the following implementation
ip="Plan a meal for breakfast, lunch, snack, and dinner within a budget of 100 Euros."
await Console(
    #await team.run()
    # team.run(input="Plan a meal for breakfast, lunch, snack, and dinner within a budget of $50.")
    team.run_stream(task=ip)
)

---------- user ----------
Plan a meal for breakfast, lunch, snack, and dinner within a budget of 100 Euros.
---------- BudgetAgent ----------
Here's a meal plan that should fit your budget of 100 Euros:

**Breakfast** (approx. 10 Euros)

* Overnight Oats with Fresh Berries: Mix together rolled oats, milk, and a drizzle of honey
---------- BreakfastAgent ----------
Here's a meal plan that should fit your budget of 100 Euros:

**Breakfast** (approx. 10 Euros)

* Overnight Oats with Fresh Berries: Mix together rolled oats, milk, and a drizzle of honey in a jar or container. Refrigerate overnight and top with fresh berries and a sprinkle of granola in the morning.

**Lunch** (approx. 30 Euros)

* Grilled Chicken and Avocado Salad: Grill chicken breast and slice it on top of a mixed green salad with sliced avocado, cherry tomatoes, and a sprinkle of feta cheese. Drizzle with a homemade vinaigrette dressing.

**Snack** (approx. 10 Euros)

* Apple Slices with Almond Butter: Spread almond but

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Plan a meal for breakfast, lunch, snack, and dinner within a budget of 100 Euros.', type='TextMessage'), TextMessage(source='BudgetAgent', models_usage=RequestUsage(prompt_tokens=3723, completion_tokens=49), content="Here's a meal plan that should fit your budget of 100 Euros:\n\n**Breakfast** (approx. 10 Euros)\n\n* Overnight Oats with Fresh Berries: Mix together rolled oats, milk, and a drizzle of honey", type='TextMessage'), TextMessage(source='BreakfastAgent', models_usage=RequestUsage(prompt_tokens=2693, completion_tokens=277), content="Here's a meal plan that should fit your budget of 100 Euros:\n\n**Breakfast** (approx. 10 Euros)\n\n* Overnight Oats with Fresh Berries: Mix together rolled oats, milk, and a drizzle of honey in a jar or container. Refrigerate overnight and top with fresh berries and a sprinkle of granola in the morning.\n\n**Lunch** (approx. 30 Euros)\n\n* Grilled Chicken and Avocado Salad

Expected output should show the following:

**Agents' Initial Suggestions:** Each agent provides meal recommendations (e.g., breakfast, lunch, dinner, snack) with corresponding costs.

**Budget Validation:** The Budget_Agent uses the budget_checker_tool to validate if the total cost fits within the budget. If it doesn't, the agent provides a clear message explaining how much the budget is exceeded.

**Adjustments by Agents:** If the budget is exceeded, agents revise their recommendations to bring the total cost within the budget.

**Task Completion:** The system stops after reaching the maximum number of allowed messages, with a summary of the interactions.

- Question: What are the drawbacks of this approach?

Answer:

### ***What You Should Remember:***

1. **Multi-Agent Systems** allow specialized agents to collaboratively solve tasks, with each agent focusing on a specific role (e.g., meal planning, budget validation).

2. **RoundRobinGroupChat** ensures that agents interact sequentially, providing structured communication and ensuring every agent contributes in a predefined order.

3. **SelectorGroupChat** dynamically selects agents based on the task's context, enabling efficient and context-aware delegation of responsibilities.

5. **Assistant Agents** are modular components with specialized roles (e.g., Breakfast Agent, Snack Agent) that generate context-specific responses and recommendations.


7. **System Messages and Descriptions** guide agent behavior, specifying their focus (e.g., dietary preferences, time constraints, balanced nutrition) and ensuring clarity in their contributions.

8. **Building a Multi-Agent System** involves:
    - **Agent Setup:** Defining agents with specific roles and integrating tools like the Budget Checker.
    - **Coordination Mechanisms:** Using methods like `RoundRobinGroupChat` to manage agent interactions.